In [1]:
!pip install cmake 'gym[atari]' scipy

     |████████████████████████████████| 21.5 MB 6.1 MB/s eta 0:00:01     |█████████████████▍              | 11.7 MB 6.1 MB/s eta 0:00:02
     |████████████████████████████████| 1.5 MB 28.9 MB/s eta 0:00:01
     |████████████████████████████████| 28.5 MB 14.6 MB/s eta 0:00:01
     |████████████████████████████████| 1.6 MB 32.3 MB/s eta 0:00:01
  Created wheel for gym: filename=gym-0.21.0-py3-none-any.whl size=1616821 sha256=72a00d48fc3dca179b35be30fc09bd255fb817f15dad27d2bb644430875e1d22
  Stored in directory: /root/.cache/pip/wheels/b3/50/6c/0a82c1358b4da2dbd9c1bb17e0f89467db32812ab236dbf6d5
Successfully built gym
You should consider upgrading via the '/usr/local/bin/python -m pip install --upgrade pip' command.


In [11]:
import gym

env = gym.make("Taxi-v3").env

In [12]:
env.reset() # reset environment to a new, random state
env.render()

print("Action Space {}".format(env.action_space))
print("State Space {}".format(env.observation_space))

+---------+
|R: | : :G|
| : | : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+

Action Space Discrete(6)
State Space Discrete(500)


In [13]:
import numpy as np
q_table = np.zeros([env.observation_space.n, env.action_space.n])

In [14]:
%%time
"""Training the agent"""

import random
from IPython.display import clear_output

# Hyperparameters
alpha = 0.1
gamma = 0.6
epsilon = 0.1

# For plotting metrics
all_epochs = []
all_penalties = []

for i in range(1, 100001):
    state = env.reset()

    epochs, penalties, reward, = 0, 0, 0
    done = False
    
    while not done:
        if random.uniform(0, 1) < epsilon:
            action = env.action_space.sample() # Explore action space
        else:
            action = np.argmax(q_table[state]) # Exploit learned values

        next_state, reward, done, info = env.step(action) 
        
        old_value = q_table[state, action]
        next_max = np.max(q_table[next_state])
        
        new_value = (1 - alpha) * old_value + alpha * (reward + gamma * next_max)
        q_table[state, action] = new_value

        if reward == -10:
            penalties += 1

        state = next_state
        epochs += 1
        
    if i % 100 == 0:
        clear_output(wait=True)
        print(f"Episode: {i}")

print("Training finished.\n")

Episode: 100000
Training finished.

CPU times: user 50 s, sys: 4.73 s, total: 54.7 s
Wall time: 53 s


In [22]:
next_state, reward, done, info

(410, 20, True, {'prob': 1.0})

In [24]:
old_value, new_value

(19.99999999999999, 19.99999999999999)

In [28]:
q_table[308, 0]

-1.450240000000004

In [29]:
epochs, penalties, reward

(14, 0, 20)

In [19]:
state = env.encode(3, 0, 2, 0) # (taxi row, taxi column, passenger index, destination index)
print("State:", state)

env.s = state
env.render()

State: 308
+---------+
|R: | : :G|
| : | : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+
  (Dropoff)


In [20]:
# display the action values for South, North, East, West, Pickup, Dropoff
q_table[state]

array([ -1.45024   ,  -2.12208638,  -1.87014402,  -1.870144  ,
       -10.87014328, -10.87014358])

In [21]:
"""Evaluate agent's performance after Q-learning"""

total_epochs, total_penalties = 0, 0
episodes = 100

for _ in range(episodes):
    state = env.reset()
    epochs, penalties, reward = 0, 0, 0
    
    done = False
    
    while not done:
        action = np.argmax(q_table[state])
        state, reward, done, info = env.step(action)

        if reward == -10:
            penalties += 1

        epochs += 1

    total_penalties += penalties
    total_epochs += epochs

print(f"Results after {episodes} episodes:")
print(f"Average timesteps per episode: {total_epochs / episodes}")
print(f"Average penalties per episode: {total_penalties / episodes}")

Results after 100 episodes:
Average timesteps per episode: 13.09
Average penalties per episode: 0.0
